In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od

In [5]:
kaggle_username = "amithsurasani"
kaggle_key = "e366f530535dfa85184b503e42c7ce1c"

In [6]:
name_of_dataset = "https://www.kaggle.com/datasets/felipekitamura/spr-x-ray-age-and-gender-dataset"
download_path = "C:/Users/amith/Documents/Datasets"

od.download(dataset_id_or_url=name_of_dataset,data_dir=download_path,)
print()
print("Download over!")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading spr-x-ray-age-and-gender-dataset.zip to C:/Users/amith/Documents/Datasets\spr-x-ray-age-and-gender-dataset


100%|██████████| 15.6G/15.6G [09:50<00:00, 28.4MB/s]


KeyboardInterrupt: 